In [158]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [159]:
import os
import shutil
import numpy as np
import pandas as pd
from keras.preprocessing.image import load_img, img_to_array
from keras import models, layers
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

In [160]:
file_dir = '/content/drive/MyDrive/Colab Notebooks/Patrones/File'

In [161]:
labels_dir = '/content/drive/MyDrive/Colab Notebooks/Patrones/File/labels.csv'
labels = pd.read_csv(labels_dir)[0:50]
labels.columns

Index(['image_id', 'patient_id', 'camera', 'patient_age', 'comorbidities',
       'diabetes_time_y', 'insuline', 'patient_sex', 'exam_eye', 'diabetes',
       'nationality', 'optic_disc', 'vessels', 'macula', 'DR_SDRG', 'DR_ICDR',
       'focus', 'iluminaton', 'image_field', 'artifacts',
       'diabetic_retinopathy', 'macular_edema', 'scar', 'nevus', 'amd',
       'vascular_occlusion', 'hypertensive_retinopathy', 'drusens',
       'hemorrhage', 'retinal_detachment', 'myopic_fundus',
       'increased_cup_disc', 'other', 'quality'],
      dtype='object')

In [162]:
labels.head()

,image_id,patient_id,camera,patient_age,comorbidities,diabetes_time_y,insuline,patient_sex,exam_eye,diabetes,...,amd,vascular_occlusion,hypertensive_retinopathy,drusens,hemorrhage,retinal_detachment,myopic_fundus,increased_cup_disc,other,quality
0,img00001,1,Canon CR,48.0,diabetes1,12,yes,1,1,yes,...,0,0,0,0,0,0,0,1,0,Adequate
1,img00002,1,Canon CR,48.0,diabetes1,12,yes,1,2,yes,...,0,0,0,0,0,0,0,1,0,Adequate
2,img00003,2,Canon CR,18.0,diabetes1,7,yes,2,1,yes,...,0,0,0,0,0,0,0,0,0,Adequate
3,img00004,2,Canon CR,18.0,diabetes1,7,yes,2,2,yes,...,0,0,0,0,0,0,0,0,0,Adequate
4,img00005,3,Canon CR,22.0,diabetes1,11,yes,1,1,yes,...,0,0,0,0,0,0,0,0,0,Adequate


In [163]:
img_dir = []
for img_id in labels['image_id']:
  img_id = '/content/drive/MyDrive/Colab Notebooks/Patrones/File/img/'+img_id+'.jpg'
  img_dir.append(plt.imread(img_id))

In [164]:
# Definimos un dataframe con solo características
df_feat = labels[['diabetic_retinopathy','macular_edema', 'scar', 'nevus', 'amd','vascular_occlusion', 'hypertensive_retinopathy', 'drusens','hemorrhage', 'retinal_detachment', 'myopic_fundus', 'increased_cup_disc', 'other']]
df_feat.head()

,diabetic_retinopathy,macular_edema,scar,nevus,amd,vascular_occlusion,hypertensive_retinopathy,drusens,hemorrhage,retinal_detachment,myopic_fundus,increased_cup_disc,other
0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0


In [165]:
# Definimos un dataframe con solo la variable objetivo
#df_target = pd.DataFrame(img_dir, columns=["img_dir"])
#df_target.head()

# Convert the list of images to a list of strings
img_paths = []
for img in img_dir:
  img_paths.append(str(img))

# Create the DataFrame with the correct data type
df_target = pd.DataFrame(img_paths, columns=["img_dir"])

In [166]:
etiquetas_entrenamiento, etiquetas_prueba, imagenes_entrenamiento, imagenes_prueba = train_test_split(df_feat, img_dir, test_size=0.30, random_state=101)

In [167]:
# Definir la función de preprocesamiento de imágenes
def preprocesar_imagen(imagen):
  # Normalizar los valores de píxeles entre 0 y 1
  imagen = tf.cast(imagen, tf.float32) / 255.0
  # Redimensionar la imagen al tamaño de entrada de la red neuronal
  imagen = tf.image.resize(imagen, (224, 224))
  return imagen

In [168]:
# Aplicar la función de preprocesamiento a las imágenes de entrenamiento y prueba
imagenes_entrenamiento_preprocesadas = [preprocesar_imagen(imagen) for imagen in imagenes_entrenamiento]
imagenes_prueba_preprocesadas = [preprocesar_imagen(imagen) for imagen in imagenes_prueba]

In [169]:
# Convertir las etiquetas a formato categórico
etiquetas_entrenamiento_categoricas = keras.utils.to_categorical(etiquetas_entrenamiento)
etiquetas_prueba_categoricas = keras.utils.to_categorical(etiquetas_prueba)

# Definir la arquitectura de la red neuronal
modelo = keras.Sequential([
  # Capa de convolución 1
  keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
  # Capa de agrupación máxima 1
  keras.layers.MaxPooling2D(2, 2),

  # Capa de convolución 2
  keras.layers.Conv2D(64, (3, 3), activation='relu'),
  # Capa de agrupación máxima 2
  keras.layers.MaxPooling2D(2, 2),

  # Capa de convolución 3
  keras.layers.Conv2D(128, (3, 3), activation='relu'),
  # Capa de agrupación máxima 3
  keras.layers.MaxPooling2D(2, 2),

  # Capa de aplanamiento
  keras.layers.Flatten(),

  # Capa densa 1
  keras.layers.Dense(64, activation='relu'),

  # Capa de salida
  keras.layers.Dense(10, activation='softmax')
])

# Compilar la red neuronal
modelo.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# Entrenar la red neuronal
modelo.fit(imagenes_entrenamiento_preprocesadas, etiquetas_entrenamiento_categoricas, epochs=10)

# Evaluar la precisión del modelo en el conjunto de prueba
precision = modelo.evaluate(imagenes_prueba_preprocesadas, etiquetas_prueba_categoricas)
print('Precisión:', precision[1])

ValueError: Data cardinality is ambiguous:
  x sizes: 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224
  y sizes: 35
Make sure all arrays contain the same number of samples.